# Setup

In [14]:
import numpy as np
import pandas as pd
import sklearn
import os
import boto3

In [3]:
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]
AWS_DEFAULT_REGION = os.environ["AWS_DEFAULT_REGION"]

In [5]:
os.environ['AWS_DEFAULT_REGION']

'eu-west-3'

In [10]:
s3_client = boto3.client('s3', region_name=AWS_DEFAULT_REGION,
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
bucket_name = "our-recipe-recipes-table-test"
file_name = "recipes.csv"

# Load the dataset

In [20]:
s3_client.download_file(bucket_name, file_name, "data/recipes.csv")

In [21]:
df = pd.read_csv("data/recipes.csv")
df.head()

,id,title,nutriScore,mealType,kcal,sodium,sugars,carbs,protein,fat,saturates,fibre
0,-297529017418066210,Coffee Protein Smoothie,60.00,Snack,356.40,0.14072,25.94,34.59,34.21,10.16,5.15,2.60
1,2984524080050943643,Sear tuna and potatoes,69.09,Dinner,368.50,1.55038,10.12,40.60,26.65,8.26,6.10,7.54
2,-134860378839050690,Sear tuna and potatoes,69.09,Dinner,368.50,1.55038,10.12,40.60,26.65,8.26,6.10,7.54
3,-6989815825829559279,Burrito Jars,81.82,Lunch,505.25,0.28775,6.25,15.25,54.50,21.50,3.25,8.75
4,1662862941331579196,NEW APPLE,72.73,Breakfast,115.00,0.00200,23.00,25.00,0.00,0.00,0.00,5.00


In [19]:
X = df.iloc[:, 2:].to_numpy()